In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder

from keras import layers
from keras import models
from keras import losses
from keras.utils import to_categorical

In [3]:
dateColumnNames = [
    'contact_date',
    'Glycemie_der_date',
    'HbA1c_der_date',
    'der_date_poids',
    'der_date_taille',
    'first_contact_date'
]

dfView = pd.read_csv('PatientsHTA.zip',nrows=1)
df = pd.read_csv('PatientsHTA.zip',engine='c',parse_dates=dateColumnNames)

# Suppression des lignes trop peu nombreuses

Nous souhaitons faire un apprentissage en utilisant la dimension temporelle comme filtre pour le CNN. Pour ça il faut donc que nous ayons plusieurs entrées. Avant de commencer à traîter les données, nous supprimons toutes les personnes qui n'ont pas rendu visite assez souvent à leur médecin. Ainsi, par le biais de ```person_id```, nous avons choisi arbitrairement que pour être utile à l'apprentissage, il faut au moins 3 visites par patients, soient toutes les lignes dont le ```person_id```est contenu plus de 3 fois dans tout le jeu de données.

In [4]:
valueCounts = df.person_id.value_counts()
dfEnought = df[df.person_id.isin(valueCounts[valueCounts.values >= 3].index)]

# Suppression des colonnes innutiles

## Suppression de la colonne ```age_now```

Nous pouvons supprimer la colonne ```age_now``` car les données qu'elle contient sont identiques à celles de la colonne ```year_of_birth```.



In [5]:
dfWithoutAgeNow = dfEnought.drop('Age_now', axis='columns')

## Suppression de la colonne ```contact_id```

En effet, la colonne ```contact_id``` ne représente aucun intérêt pour l'apprentissage car elle ne contient aucun information à même d'influer sur la prédiction.

In [6]:
dfWithoutContactID = dfWithoutAgeNow.drop('contact_id',axis='columns')

## Suppression des noms de médicaments

In [7]:
dfGroupedByMoleculeLabel = dfWithoutContactID.groupby('product_atc_code')[['molecule_label','short_name','long_name','Classe','product_atc']].count()
dfGroupedByMoleculeLabel

,molecule_label,short_name,long_name,Classe,product_atc
product_atc_code,,,,,
C02AC06,4706,4706,4706,4706,4706
C03BX03,977,977,977,977,977
C03CA01,1002,1002,1002,1002,1002
C03DA01,215,215,215,215,215
C03DA04,33,33,33,33,33
C03EA04,75,75,75,75,75
C03EB01,14,14,14,14,14
C07AA05,2,2,2,2,2
C07AB03,7026,7026,7026,7026,7026


Nous voyons qu'il existe différentes colonnes dont le but est de désigner le médicament prescrit lors de la visite, or nous n'avons besoin que d'une seule colonne garder cette information. De ce fait, nous avons choisi de garder ```product_atc_code```.

In [8]:
dropColumnNames = dfGroupedByMoleculeLabel.columns.to_list()
dfWithATCCode = dfWithoutContactID.drop(dropColumnNames, axis='columns')
dfWithATCCode

,person_id,specialty_label,contact_date,cip,dosage_1,dose_1,dose_2,product_atc_code,box,quantity,...,HbA1c_der_date,HbA1c_der_mesure,gender_code,Age_presc,year_of_birth,Poids,der_date_poids,Taille,der_date_taille,first_contact_date
1,263659.0,Médecin généraliste,2014-09-24,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,96.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
2,263659.0,Médecin généraliste,2014-09-12,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,96.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
3,263659.0,Médecin généraliste,2015-03-26,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,97.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
4,263659.0,Médecin généraliste,2015-06-05,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,97.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
6,25182917.0,Médecin généraliste,2013-03-27,3.400960e+12,300.0,16800.0,56.0,C09XA02,2.0,1.00,...,NaT,NaN,M,63.0,1950.0,NaN,NaT,NaN,NaT,2011-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50627,18889430.0,Médecin généraliste,2013-02-01,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.50,...,2017-05-15,8.01,M,85.0,1928.0,NaN,NaT,NaN,NaT,2007-12-02
50629,2222336.0,Médecin généraliste,2013-07-01,3.400940e+12,500.0,15000.0,30.0,C03CA01,2.0,0.25,...,NaT,NaN,F,78.0,1935.0,NaN,NaT,NaN,NaT,2006-02-23
50630,24755853.0,Médecin généraliste,2013-01-22,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.50,...,NaT,NaN,M,87.0,1926.0,NaN,NaT,NaN,NaT,2011-05-01
50631,11363518.0,Médecin généraliste,2013-01-24,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.25,...,NaT,NaN,M,80.0,1933.0,NaN,NaT,NaN,NaT,1998-01-27


## Suppression des colonnes ```'*der*'```

Les colonnes ```'*der*'``` contiennent la dernière donnée. Cette donnée peut être récupérée grâce à la date de la visite et aux valeurs mesurées. Par exemple, il n'est pas nécessaire d'avoir une colonne ```der_date``` ou ```der_mesure```. Les données de ces deux types de colonnes peuvent être récupérées grâce à la ligne qui correspond à la dernière date de la mesure, que nous pouvons trouver grâce à la colonne ```contact_date```.

In [9]:
derColumnNames = []

for c in dfWithATCCode.columns:
    if ('der_date' in c) or ('der_mesure' in c):
        derColumnNames.append(c)

dfWithoutDer = dfWithATCCode.drop(derColumnNames,axis='columns')

## Suppression des colonnes ```Taille``` et ```Poids```

In [82]:
print(f"Taille: {dfWithoutDer.Taille.isnull().sum()}/{len(dfWithoutDer.Taille)} valeurs nulles (={dfWithoutDer.Taille.isnull().sum()/len(dfWithoutDer.Taille)*100}%)")

print(f"Poids: {dfWithoutDer.Poids.isnull().sum()}/{len(dfWithoutDer.Poids)} valeurs nulles (={dfWithoutDer.Poids.isnull().sum()/len(dfWithoutDer.Poids)*100}%)".format())

Taille: 20328/40587 valeurs nulles (=50.085002587035255%)
Poids: 12982/40587 valeurs nulles (=31.985611156281568%)


Nous voyons qu'il y a beaucoup trop de valeur nulles. Ces deux colonnes semblent donc difficilement exploitable. Nous pouvons cependant vérifier si pour les patients toutes les valeurs sont à nulles ou s'il n'existe que quelques entrées à nulle par patient mais qu'il y en a beaucoup. Dans ce cas nous pourrions enlever les lignes contenant des valeurs nulles, ou trouver un moyen d'attribuer une valeur à la place de Nan

In [70]:
dfPersonIdIndex = dfWithoutDer.set_index('person_id',drop=True).sort_index()
dfTPGroupBy = dfPersonIdIndex.groupby('person_id')

In [87]:
maybeUseful = 0
valeurNulle = 0
for i,_ in dfTPGroupBy:
    if dfPersonIdIndex.loc[i].Taille.isnull().sum() > 0:
        valeurNulle += 1
        if dfPersonIdIndex.loc[i].Taille.isnull().sum() < len(dfPersonIdIndex.loc[i].Taille):
            maybeUseful += 1

print(f"Taille: {maybeUseful} / {poidsNull} utilisables")

maybeUseful = 0
valeurNulle = 0
c = 0
for i,_ in dfTPGroupBy:
    if dfPersonIdIndex.loc[i].Poids.isnull().sum() > 0:
        valeurNulle += 1
        if dfPersonIdIndex.loc[i].Poids.isnull().sum() < len(dfPersonIdIndex.loc[i].Poids):
            maybeUseful += 1

print(f"Poids: {maybeUseful} / {poidsNull} utilisables")

Taille: 0 / 2945 utilisable
Poids: 0 / 2945 utilisable


In [92]:
tailleNan = 0
poidsNan = 0
oneOfBoth = 0
bothNan = 0
totalEntries = 0
for i,_ in dfTPGroupBy:
    totalEntries += 1
    hasTailleNan = False
    hasPoidsNan = False
    if dfPersonIdIndex.loc[i].Taille.isnull().sum() != 0:
        tailleNan += 1
        hasTailleNan = True
    if dfPersonIdIndex.loc[i].Poids.isnull().sum() != 0:
        poidsNan += 1
        hasPoidsNan = True
    if hasTailleNan or hasPoidsNan:
        oneOfBoth += 1
    if hasTailleNan and hasPoidsNan:
        bothNan += 1
print(" --- Statistique par Utilisateur --- ")
print(f"{tailleNan} / {totalEntry} ({tailleNan/totalEntries*100:.2f}%) des utilisateurs ont une valeur nulle pour la taille")
print(f"{poidsNan} / {totalEntry} ({poidsNan/totalEntries*100:.2f}%) des utilisateurs ont une valeur nulle pour le poids")
print(f"{oneOfBoth} / {totalEntry} ({oneOfBoth/totalEntries*100:.2f}%) des utilisateurs ont une valeur nulle pour la taille ou le poids")
print(f"{bothNan} / {totalEntry} ({bothNan/totalEntries*100:.2f}%) des utilisateurs ont les deux valeurs nulle pour la taille ou le poids")



 --- Statistique par Utilisateur --- 
2945 / 5464 (53.90%) des utilisateurs ont une valeur nulle pour la taille
2029 / 5464 (37.13%) des utilisateurs ont une valeur nulle pour le poids
2949 / 5464 (53.97%) des utilisateurs ont une valeur nulle pour la taille ou le poids
2025 / 5464 (37.06%) des utilisateurs ont les deux valeurs nulle pour la taille ou le poids


Nous concluons de l'analyse de ces données que soit toutes les valeurs de poids et de tailles sont entrées, soit aucunes. Cela rend ces informations innexploitables et nous supprimons donc les colonnes

In [94]:
dfWithoutPT = dfWithoutDer.drop(['Taille', 'Poids'],axis='columns')

## Suppressions diverses

Enfin, certaines colonnes n'apportent pas d'informations supplémentaires, nous choisissons de toutes les supprimer ici

In [95]:
dfFinal = dfWithoutPT.drop(['cip','box'],axis='columns')

# Traîtement des données

## Conversion des données

### Ajout du temps entre chaque visite (ce que l'on veut prédire)

Nous créons d'abord la colonne ```wait_time``` pour qu'elle ait le type de donnée ```deltatime```. nous itèrerons plus tard sur chaque valeur de cette colonne pour lui enlever la valeur précédante pour chaque utilisateur, et ainsi avoir l'intervalle de temps entre chaque visite

In [96]:
wait_time = dfFinal.contact_date - dfFinal.first_contact_date

dfWithTime = dfFinal.drop('first_contact_date',axis='columns')
dfWithTime['wait_time'] = wait_time

In [97]:
dfWithTime

,person_id,specialty_label,contact_date,dosage_1,dose_1,dose_2,product_atc_code,quantity,frequency_label,duration,...,Traitement_Insulines_dep_201701,Pulse,Tension Diastolique,Tension Systolique,Glycemie_prescription,HbA1c_prescription,gender_code,Age_presc,year_of_birth,wait_time
1,263659.0,Médecin généraliste,2014-09-24,20.0,240.0,24.0,C03CA01,4.00,Jour,NaN,...,NaN,70.0,110.0,60.0,NaN,NaN,M,96.0,1918.0,6019 days
2,263659.0,Médecin généraliste,2014-09-12,20.0,240.0,24.0,C03CA01,4.00,Jour,NaN,...,NaN,70.0,118.0,70.0,NaN,NaN,M,96.0,1918.0,6007 days
3,263659.0,Médecin généraliste,2015-03-26,20.0,240.0,24.0,C03CA01,4.00,Jour,NaN,...,NaN,60.0,120.0,70.0,NaN,NaN,M,97.0,1918.0,6202 days
4,263659.0,Médecin généraliste,2015-06-05,20.0,240.0,24.0,C03CA01,4.00,Jour,NaN,...,NaN,60.0,120.0,70.0,NaN,NaN,M,97.0,1918.0,6273 days
6,25182917.0,Médecin généraliste,2013-03-27,300.0,16800.0,56.0,C09XA02,1.00,Jour,84.0,...,NaN,64.0,145.0,85.0,NaN,NaN,M,63.0,1950.0,723 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50627,18889430.0,Médecin généraliste,2013-02-01,500.0,15000.0,30.0,C03CA01,0.50,Jour,28.0,...,NaN,60.0,120.0,70.0,NaN,NaN,M,85.0,1928.0,1888 days
50629,2222336.0,Médecin généraliste,2013-07-01,500.0,15000.0,30.0,C03CA01,0.25,Jour,240.0,...,NaN,72.0,130.0,70.0,NaN,NaN,F,78.0,1935.0,2685 days
50630,24755853.0,Médecin généraliste,2013-01-22,500.0,15000.0,30.0,C03CA01,0.50,Jour,56.0,...,NaN,73.0,141.0,71.0,NaN,NaN,M,87.0,1926.0,632 days
50631,11363518.0,Médecin généraliste,2013-01-24,500.0,15000.0,30.0,C03CA01,0.25,Jour,120.0,...,NaN,72.0,139.0,70.0,NaN,NaN,M,80.0,1933.0,5476 days


### Encodage des valeurs non numériques

In [98]:
specialtyEncoder = LabelEncoder()
ATCCodeEncoder = LabelEncoder()
frequencyLabelEncoder = LabelEncoder()
traitementAutresLabelEncoder = LabelEncoder()
traitementInsulineLabelEncoder = LabelEncoder()
genderEncoder = LabelEncoder()

dfWithTime.specialty_label = specialtyEncoder.fit_transform(dfWithTime.specialty_label)
dfWithTime.product_atc_code = ATCCodeEncoder.fit_transform(dfWithTime.product_atc_code)
dfWithTime.frequency_label = frequencyLabelEncoder.fit_transform(dfWithTime.frequency_label.astype(str))
dfWithTime.Traitement_Autres_A10_dep_201701 = traitementAutresLabelEncoder.fit_transform(dfWithTime.Traitement_Autres_A10_dep_201701.astype(str))
dfWithTime.Traitement_Insulines_dep_201701 = traitementInsulineLabelEncoder.fit_transform(dfWithTime.Traitement_Insulines_dep_201701.astype(str))
dfWithTime.gender_code = ATCCodeEncoder.fit_transform(dfWithTime.gender_code)


### Conversion en ```TimeSeries```

Nous définissons simplement le nouvel index comme la colonne donnant l'intervalle de temps entre chaque visite.

In [99]:
ts = dfWithTime.set_index(['person_id','contact_date']).sort_index()
ts

specialty_label  dosage_1  dose_1  dose_2  \
person_id  contact_date                                              
291.0      2016-10-28                  2       2.5    75.0    30.0   
           2016-11-04                  2       2.5    75.0    30.0   
           2016-11-21                  2       2.5    75.0    30.0   
           2017-01-16                  2       2.5    75.0    30.0   
           2017-11-20                  2       2.5    75.0    30.0   
...                                  ...       ...     ...     ...   
35576034.0 2017-11-28                  2      40.0    90.0     0.0   
           2018-02-26                  2      40.0    90.0     0.0   
35849563.0 2017-01-12                  0     300.0  7200.0    24.0   
           2017-12-18                  0     300.0  7200.0    24.0   
           2018-05-01                  0     300.0  7200.0    24.0   

                         product_atc_code  quantity  frequency_label  \
person_id  contact_date                                                
291.0      2016-10-28                  10       2.0                0   
           2016-11-04                  10       1.0                0   
           2016-11-21                  10       1.0                0   
           2017-01-16                  10       1.0                0   
           2017-11-20                  10       1.0                0   
...                                   ...       ...              ...   
35576034.0 2017-11-28                  30       1.0                0   
           2018-02-26                  30       1.0                0   
35849563.0 2017-01-12                  14       1.0                0   
           2017-12-18                  14       1.0                0   
           2018-05-01                  14       1.0                0   

                         duration  Traitement_Autres_A10_dep_201701  \
person_id  contact_date                                               
291.0      2016-10-28        84.0                                 0   
           2016-11-04        84.0                                 0   
           2016-11-21        84.0                                 0   
           2017-01-16        84.0                                 0   
           2017-11-20        84.0                                 0   
...                           ...                               ...   
35576034.0 2017-11-28        84.0                                 0   
           2018-02-26        84.0                                 0   
35849563.0 2017-01-12        24.0                                 1   
           2017-12-18        24.0                                 1   
           2018-05-01        48.0                                 1   

                         Traitement_Insulines_dep_201701  Pulse  \
person_id  contact_date                                           
291.0      2016-10-28                                  0   72.0   
           2016-11-04                                  0   68.0   
           2016-11-21                                  0   84.0   
           2017-01-16                                  0   84.0   
           2017-11-20                                  0   92.0   
...                                                  ...    ...   
35576034.0 2017-11-28                                  1   68.0   
           2018-02-26                                  1   64.0   
35849563.0 2017-01-12                                  1  102.0   
           2017-12-18                                  1  100.0   
           2018-05-01                                  1   80.0   

                         Tension Diastolique  Tension Systolique  \
person_id  contact_date                                            
291.0      2016-10-28                  130.0                60.0   
           2016-11-04                  142.0                65.0   
           2016-11-21                  140.0                75.0   
           2017-01-16                  12

### Attribution des bonnes valeurs de ```time_wait```

In [100]:
tsWithTime = ts
for i,_ in ts.groupby('person_id'):
    for j in range(len(ts.loc[i].wait_time)-1,0,-1):
        tsWithTime.loc[i].wait_time[j] = ts.loc[i].wait_time[j] - ts.loc[i].wait_time[j-1]
    tsWithTime.loc[i].wait_time[0] = pd.Timedelta(0)

In [103]:
tsWithTime.wait_time

person_id   contact_date
291.0       2016-10-28       0 days
            2016-11-04       7 days
            2016-11-21      17 days
            2017-01-16      56 days
            2017-11-20     308 days
                             ...   
35576034.0  2017-11-28     263 days
            2018-02-26      90 days
35849563.0  2017-01-12       0 days
            2017-12-18     340 days
            2018-05-01     134 days
Name: wait_time, Length: 40587, dtype: timedelta64[ns]

## Traîtement des valeurs ```Nan```

```VOIR S'IL Y A ENCORE DES NAN ET LES TRAÎTER```

# Prédiction

## Création des données d'entraînement/test

In [104]:
yColumNames = ['product_atc_code','wait_time']

xDf = tsWithTime.drop(yColumNames,axis='columns')

# Pour l'instant on ne prédit que l'atc code et c'est déjà un sacré bordel
yDf = tsWithTime.product_atc_code#loc[:,yColumNames]

xList,yList = [],[]
sliceNumber = 3

for i,_ in ts.groupby('person_id'):
    # Mme ZERHAOUI a dit qu'il fallait transposer, je transpose
    xList.append(xDf.loc[i][:sliceNumber].to_numpy().astype('float32').transpose())
    yList.append(yDf.loc[i][:sliceNumber].to_numpy().astype('float32'))

xData = np.array(xList).reshape((len(xList),xList[0].shape[0],sliceNumber))
yData = np.array(yList).reshape((sliceNumber,len(yList)))#,yList[0].shape[1]))

In [105]:
print(f"{xData.shape}, {yData.shape}")

(5464, 17, 3), (3, 5464)


In [106]:
# On crée un jeu d'entraînement et de test :
#  - l'entraînement se fait sur 80% du jeu de donnée total
#  - le test se fait donc sur 20%

trainUse = int(xData.shape[0] * 80 / 100)
testUse = xData.shape[0] - trainUse

xTrain, xTest, yTrain, yTest = xData[:trainUse],xData[-testUse:],yData[:,:trainUse],yData[:,-testUse:]
print(f"train: {trainUse}   test: {testUse}")

train: 4371   test: 1093


## Création du modèle

In [107]:
print(f"{xTrain.shape}, {yTrain.shape}")

(4371, 17, 3), (3, 4371)


In [108]:
# Alors c'est totalement au hasard, j'ai essayé de suivre ce qu'il y avait dans la partie 'keras exemple' du lien que farida nous a donné :
# cf : https://towardsdatascience.com/how-to-use-convolutional-neural-networks-for-time-series-classification-56b1b0a07a57
model = models.Sequential()
kernelNumber = 20
model.add(layers.Conv1D(kernelNumber,17,activation='tanh',input_shape=(xTrain.shape[1],sliceNumber)))
model.add(layers.MaxPool1D(1))
model.add(layers.Flatten())
model.add(layers.Dense(yData.max()+1,activation='sigmoid'))

In [109]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1, 20)             1040      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 20)             0         
_________________________________________________________________
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 33)                693       
Total params: 1,733
Trainable params: 1,733
Non-trainable params: 0
_________________________________________________________________


In [110]:
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

In [111]:
# pour utiliser sparse cross entropy apparement il faut des valeurs entre 0 et 1 pourtant sur ce lien il l'utilise pour prédire les données mniste entre 0 et 9 'https://www.machinecurve.com/index.php/2019/10/06/how-to-use-sparse-categorical-crossentropy-in-keras/'
for i in range(3):
    print(f"--- Training: {i+1}/{yTrain.shape[0]} ---")
    model.fit(xTrain,yTrain[i],epochs=20, validation_data=(xTest,yTest[i]))

--- Training: 1/3 ---
Epoch 1/20
137/137 [==============================] - 1s 4ms/step - loss: 9.7860 - sparse_categorical_accuracy: 0.0151 - val_loss: 10.1900 - val_sparse_categorical_accuracy: 0.0165
Epoch 2/20
137/137 [==============================] - 1s 4ms/step - loss: 9.7860 - sparse_categorical_accuracy: 0.0151 - val_loss: 10.1900 - val_sparse_categorical_accuracy: 0.0165
Epoch 3/20
137/137 [==============================] - 0s 3ms/step - loss: 9.7860 - sparse_categorical_accuracy: 0.0151 - val_loss: 10.1900 - val_sparse_categorical_accuracy: 0.0165
Epoch 4/20
137/137 [==============================] - 0s 3ms/step - loss: 9.7860 - sparse_categorical_accuracy: 0.0151 - val_loss: 10.1900 - val_sparse_categorical_accuracy: 0.0165
Epoch 5/20
137/137 [==============================] - 1s 4ms/step - loss: 9.7860 - sparse_categorical_accuracy: 0.0151 - val_loss: 10.1900 - val_sparse_categorical_accuracy: 0.0165
Epoch 6/20
137/137 [==============================] - 0s 3ms/step - loss:

In [112]:
model.predict(xTest[5].reshape((1,17,3)))

array([[0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
        0.]], dtype=float32)